In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
# Leemos todos

df_steam_juegos = pd.read_parquet('steam_games_listo.parquet')

df_user_items = pd.read_parquet('df_user_items.parquet')

df_user_reviews = pd.read_parquet('df_user_reviews_clean.parquet')



Como es costumbre vamos a ver las info de los dataframes

In [4]:
df_steam_juegos.info()
#Hago este rename porque las demás dataframe tiene item_id y yo en su
# momento puse id en este (la verdad que no recuerdo el por qué lo hice)
df_steam_juegos.rename(columns={'id': 'item_id'},inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32133 non-null  object 
 1   publisher     32133 non-null  object 
 2   genres        32133 non-null  object 
 3   app_name      32133 non-null  object 
 4   price         32133 non-null  float64
 5   early_access  32133 non-null  float64
 6   developer     32133 non-null  object 
 7   anio          32133 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.2+ MB


In [5]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   steam_id          object
 2   item_id           object
 3   item_name         object
 4   playtime_forever  int64 
dtypes: int64(1), object(4)
memory usage: 233.2+ MB


In [6]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58400 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          58400 non-null  object
 1   item_id          58400 non-null  object
 2   recommend        58400 non-null  bool  
 3   review           58400 non-null  object
 4   anio_lanzado     48471 non-null  Int64 
 5   Sentiment_Score  58400 non-null  int64 
dtypes: Int64(1), bool(1), int64(1), object(3)
memory usage: 2.8+ MB


Voy a tener que hacer varios renames, sobre todo porque quise españolizar (no volver a hacer esto). Además de cambiar los tipos de datos para que todas las tablas tengan lo mismo 

In [7]:
df_steam_juegos.rename(columns={'anio': 'year_release'},inplace=True)

In [8]:
df_user_reviews.rename(columns={'anio_lanzado': 'year_review'},inplace=True)

In [9]:
df_steam_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       32133 non-null  object 
 1   publisher     32133 non-null  object 
 2   genres        32133 non-null  object 
 3   app_name      32133 non-null  object 
 4   price         32133 non-null  float64
 5   early_access  32133 non-null  float64
 6   developer     32133 non-null  object 
 7   year_release  32133 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.2+ MB


Una vez finalizado los renames vamos a proceder a realizar los cambios necesarios para seguir

In [10]:
df_steam_juegos['item_id'] = df_steam_juegos['item_id'].astype(str)
df_steam_juegos['year_release'] = pd.to_numeric(df_steam_juegos['year_release'], errors='coerce').astype('Int64')


df_user_items["playtime_forever"] = df_user_items["playtime_forever"]/60
df_user_items['item_id'] = df_user_items['item_id'].astype(str)

df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(str)
df_user_reviews['year_review'] = pd.to_numeric(df_user_reviews['year_review'], errors='coerce').astype('Int64')

In [11]:
df_steam_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       32133 non-null  object 
 1   publisher     32133 non-null  object 
 2   genres        32133 non-null  object 
 3   app_name      32133 non-null  object 
 4   price         32133 non-null  float64
 5   early_access  32133 non-null  float64
 6   developer     32133 non-null  object 
 7   year_release  29963 non-null  Int64  
dtypes: Int64(1), float64(2), object(5)
memory usage: 2.2+ MB


In [12]:
cantidad_unicos_steam_juegos = df_steam_juegos['item_id'].nunique()
print("Cantidad de valores únicos en 'item_id' en df_steam_juegos:", cantidad_unicos_steam_juegos)

cantidad_unicos_user_items = df_user_items['item_id'].nunique()
print("Cantidad de valores únicos en 'item_id' en df_user_items:", cantidad_unicos_user_items)

cantidad_unicos_user_reviews = df_user_reviews['item_id'].nunique()
print("Cantidad de valores únicos en 'item_id' en df_user_reviews:", cantidad_unicos_user_reviews)


Cantidad de valores únicos en 'item_id' en df_steam_juegos: 32133
Cantidad de valores únicos en 'item_id' en df_user_items: 10978
Cantidad de valores únicos en 'item_id' en df_user_reviews: 3682


In [13]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   steam_id          object 
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(1), object(4)
memory usage: 233.2+ MB


In [14]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58400 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          58400 non-null  object
 1   item_id          58400 non-null  object
 2   recommend        58400 non-null  bool  
 3   review           58400 non-null  object
 4   year_review      48471 non-null  Int64 
 5   Sentiment_Score  58400 non-null  int64 
dtypes: Int64(1), bool(1), int64(1), object(3)
memory usage: 2.8+ MB


Tuve que reconvertir nuevamente los item_id en entero. Podía hacer el merge perfectamente pero no me devolvía nada así que supuse que era un problema de tipo de dato así que decidí dejarlo en int para evitar más problemas

In [15]:
# Convertir la columna 'item_id' a flotante
df_steam_juegos['item_id'] = df_steam_juegos['item_id'].astype(float)
df_user_items['item_id'] = df_user_items['item_id'].astype(float)
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(float)

# Eliminar filas con valores no finitos (NaN o inf)
df_steam_juegos = df_steam_juegos[np.isfinite(df_steam_juegos['item_id'])]
df_user_items = df_user_items[np.isfinite(df_user_items['item_id'])]
df_user_reviews = df_user_reviews[np.isfinite(df_user_reviews['item_id'])]

# Convertir la columna 'item_id' de flotante a entero para eliminar la parte decimal
df_steam_juegos['item_id'] = df_steam_juegos['item_id'].astype(int)
df_user_items['item_id'] = df_user_items['item_id'].astype(int)
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(int)


In [34]:
# merge primigenio (sirve para combinar otros después)
df_primigenio = pd.merge(df_steam_juegos, df_user_items, on='item_id', how='inner')



In [30]:
df_primigenio

In [35]:
# merge User for genre
df_ugenre = pd.merge(df_primigenio, df_user_reviews, on=['user_id', 'item_id'], how='inner')


In [36]:
df_ugenre

,item_id,publisher,genres,app_name,price,early_access,developer,year_release,user_id,steam_id,item_name,playtime_forever,recommend,review,year_review,Sentiment_Score
0,282010,Stainless Games Ltd,Action Indie Racing,Carmageddon Max Pack,9.99,0.0,stainless games ltd,1997,InstigatorAU,76561198067728137,Carmageddon Max Pack,7.766667,True,"Oh, where do i start? DOS...",<NA>,1
1,70,Valve,Action,Half-Life,9.99,0.0,valve,1998,EizanAratoFujimaki,76561198160640899,Half-Life,23.250000,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,2015,1
2,70,Valve,Action,Half-Life,9.99,0.0,valve,1998,GamerFag,76561198006592023,Half-Life,9.833333,True,If you own any half life but not this one that...,2011,0
3,70,Valve,Action,Half-Life,9.99,0.0,valve,1998,76561198020928326,76561198020928326,Half-Life,93.316667,True,this game has had me hooked for over 4 years a...,2014,1
4,70,Valve,Action,Half-Life,9.99,0.0,valve,1998,Bluegills,76561197971857001,Half-Life,1.066667,True,"This is, by far my most favourite game I have ...",2013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40092,80,Valve,Action,Counter-Strike: Condition Zero,9.99,0.0,valve,2004,76561198015050660,76561198015050660,Counter-Strike: Condition Zero,40.266667,False,作為第一個在steam入手的遊戲，老實說當初是為了1.6來順便帶回來，由於當時網路不太穩的關...,2014,1
40093,80,Valve,Action,Counter-Strike: Condition Zero,9.99,0.0,valve,2004,76561198023508728,76561198023508728,Counter-Strike: Condition Zero,0.250000,False,i caused 9/11,2014,1
40094,80,Valve,Action,Counter-Strike: Condition Zero,9.99,0.0,valve,2004,green290,76561198051829216,Counter-Strike: Condition Zero,7.366667,True,This game was suprisingly my first FPS game.11...,2015,2
40095,80,Valve,Action,Counter-Strike: Condition Zero,9.99,0.0,valve,2004,174gamecuman700kngkakak,76561198099895210,Counter-Strike: Condition Zero,1.200000,True,:v,2014,1


In [37]:
df_ugenre = df_ugenre.to_parquet('df_ugenre.parquet')

In [21]:
# merge para bestDeveloperYear
df_bdev = pd.merge(df_steam_juegos, df_user_reviews, on='item_id', how='inner')

In [38]:
df_bdev

In [25]:
df_bdev = df_bdev.to_parquet('df_bdev.parquet')

In [44]:
df_sentimiento = pd.merge(df_user_reviews,df_steam_juegos,on='item_id',how='inner')

In [45]:
df_sentimiento = df_sentimiento.to_parquet('df_sentimiento.parquet')